# How does Cython speed up Python?

## Reason 1: Interpreted -> Compiled

## Cython version of trivial function

In [1]:
%load_ext Cython

In [22]:
%%cython -n cyfoo

def cyfoo(a, b):
    return a + b

## Profiling

In [23]:
%timeit cyfoo(1, 2)

The slowest run took 13.88 times longer than the fastest. This could mean that an intermediate result is being cached.
10000000 loops, best of 3: 78.7 ns per loop


In [24]:
print("Cython integer addition speedup: {:0.1f}%".format((112. - 79.) / 112. * 100))

Cython integer addition speedup: 29.5%


In [25]:
%timeit cyfoo('a', 'b')

The slowest run took 12.63 times longer than the fastest. This could mean that an intermediate result is being cached.
10000000 loops, best of 3: 131 ns per loop


In [26]:
print("Cython string addition speedup: {:0.1f}%".format((159. - 133.) / 159. * 100))

Cython string addition speedup: 16.4%


### For simple addition, Cython version gives consistent speedup

* With all the caveats for microbenchmarks...

## We see the same `PyNumber_Add()` entry point as for interpreted Python

In [39]:
!cat $(ipython locate)/cython/cyfoo.c

/* Generated by Cython 0.25.2 */

/* BEGIN: Cython Metadata
{
    "distutils": {
        "language": "c"
    },
    "module_name": "cyfoo"
}
END: Cython Metadata */

#define PY_SSIZE_T_CLEAN
#include "Python.h"
#ifndef Py_PYTHON_H
    #error Python headers needed to compile C extensions, please install development version of Python.
#elif PY_VERSION_HEX < 0x02060000 || (0x03000000 <= PY_VERSION_HEX && PY_VERSION_HEX < 0x03020000)
    #error Cython requires Python 2.6+ or Python 3.2+.
#else
#define CYTHON_ABI "0_25_2"
#include <stddef.h>
#ifndef offsetof
  #define offsetof(type, member) ( (size_t) & ((type*)0) -> member )
#endif
#if !defined(WIN32) && !defined(MS_WINDOWS)
  #ifndef __stdcall
    #define __stdcall
  #endif
  #ifndef __cdecl
    #define __cdecl
  #endif
  #ifndef __fastcall
    #define __fastcall
  #endif
#endif
#ifndef DL_IMPORT
  #define DL_IMPORT(t) t
#endif
#ifndef DL_EXPORT
  #define DL_EXPORT(t) t
#endif
#ifndef HAVE_LONG_LONG
  #if PY_VERSION_HEX >= 0x03030000 || (

static CYTHON_INLINE size_t __Pyx_Py_UNICODE_strlen(const Py_UNICODE *u)
{
    const Py_UNICODE *u_end = u;
    while (*u_end++) ;
    return (size_t)(u_end - u - 1);
}
#else
#define __Pyx_Py_UNICODE_strlen Py_UNICODE_strlen
#endif
#define __Pyx_PyUnicode_FromUnicode(u)       PyUnicode_FromUnicode(u, __Pyx_Py_UNICODE_strlen(u))
#define __Pyx_PyUnicode_FromUnicodeAndLength PyUnicode_FromUnicode
#define __Pyx_PyUnicode_AsUnicode            PyUnicode_AsUnicode
#define __Pyx_NewRef(obj) (Py_INCREF(obj), obj)
#define __Pyx_Owned_Py_None(b) __Pyx_NewRef(Py_None)
#define __Pyx_PyBool_FromLong(b) ((b) ? __Pyx_NewRef(Py_True) : __Pyx_NewRef(Py_False))
static CYTHON_INLINE int __Pyx_PyObject_IsTrue(PyObject*);
static CYTHON_INLINE PyObject* __Pyx_PyNumber_IntOrLong(PyObject* x);
static CYTHON_INLINE Py_ssize_t __Pyx_PyIndex_AsSsize_t(PyObject*);
static CYTHON_INLINE PyObject * __Pyx_PyInt_FromSize_t(size_t);
#if CYTHON_ASSUME_SAFE_MACROS
#define __pyx_PyFloat_AsDouble(x) (PyFloat_CheckExact(x) ?



```c
static PyObject 
*__pyx_pf_5cyfoo_cyfoo(CYTHON_UNUSED PyObject *__pyx_self,
                       PyObject *__pyx_v_a,
                       PyObject *__pyx_v_b) {
[...]
  /* "cyfoo.pyx":3
 * 
 * def cyfoo(a, b):
 *     return a + b             # <<<<<<<<<<<<<<
 */
  __pyx_t_1 = PyNumber_Add(__pyx_v_a, __pyx_v_b);
 [...]
}
```

## We conclude: converting from interpreted to compiled code gives some speedup

## Reason 2: Dynamic -> Static Typing

In [63]:
def pyfac(n):
    if n <= 1:
        return 1
    return n * pyfac(n - 1)

In [64]:
%timeit pyfac(20.0)
pyfac(20.0)

The slowest run took 4.07 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 4.29 µs per loop


2.43290200817664e+18

In [65]:
%%cython

def cyfac(n):
    if n <= 1:
        return 1
    return n * cyfac(n - 1)

def cyfac_double(double n):
    if n <= 1:
        return 1.0
    return n * cyfac_double(n - 1)

In [66]:
%timeit cyfac(20.0)
cyfac(20.0)

100000 loops, best of 3: 2.2 µs per loop


2.43290200817664e+18

In [67]:
%timeit cyfac_double(20.0)
cyfac_double(20.0)

The slowest run took 5.11 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 1.35 µs per loop


2.43290200817664e+18

## Optimal Cython solution: ~40x speedup

* Optimal for *this* recursive implementation...

In [69]:
%%cython

cpdef double cyfac_double_fast(double n):
    if n <= 1:
        return 1.0
    return n * cyfac_double_fast(n - 1)

In [70]:
%timeit cyfac_double_fast(20.0)
cyfac_double_fast(20.0)

The slowest run took 10.83 times longer than the fastest. This could mean that an intermediate result is being cached.
10000000 loops, best of 3: 100 ns per loop


2.43290200817664e+18

## For the record: what about a loop-based version?

In [75]:
def pyfac_loop(n):
    r = 1.0
    for i in range(1, n+1):
        r *= i
    return r

In [76]:
%timeit pyfac_loop(20)
pyfac_loop(20)

1000000 loops, best of 3: 1.81 µs per loop


2.43290200817664e+18

In [78]:
%%cython -a

cpdef double cyfac_loop(int n):
    cdef double r = 1.0
    cdef int i
    for i in range(1, n+1):
        r *= <double>i
    return r

In [74]:
%timeit cyfac_loop(20)
cyfac_loop(20)

The slowest run took 17.30 times longer than the fastest. This could mean that an intermediate result is being cached.
10000000 loops, best of 3: 61.9 ns per loop


2.43290200817664e+18

In [77]:
print("Cython speedup factor--loop-based version: {:0.1f}".format((1.81 / 0.062)))

Cython speedup factor--loop-based version: 29.2


## Excercises / questions

* Why are we using `double` here instead of `long`?
* Why are the `pyfac_loop()` and `cyfac_loop()` versions *better* from a robustness pov?
* Write a trivial no-op function in Python and measure its performance w/ `timeit`.  Now, make a Cython no-op `def` function, and measure *it*.  How do they compare?  Conjecture why.  What does this imply for function call overhead between pure Python and Cython code?

In [79]:
def pynoop(): pass
%timeit pynoop()

10000000 loops, best of 3: 85.6 ns per loop


In [80]:
%%cython -a
def cynoop(): pass

In [81]:
%timeit cynoop()

10000000 loops, best of 3: 40.3 ns per loop
